# chat UI in fastHTML

> chat UI implemented in fastHTML

In [ ]:
#| default_exp chat_ui

In [1]:
#| hide
from nbdev.showdoc import *

## Chat App initialization

Start by creating the chat application with `FastHTML`.

In [2]:
#| export
import uvicorn
import uuid
import os
from fasthtml.common import *
from fastcore.parallel import startthread
from typing import Callable, Optional

from llmcam.fn_to_fc import capture_youtube_live_frame_and_save, ask_gpt4v_about_image_file
from llmcam.fn_to_fc import tool_schema, complete, form_msg
from llmcam.store import add_api_tools, add_function_tools, remove_tools
from llmcam.store import execute_handler_core, handler_schema
from llmcam.yolo import detect_objects
from llmcam.dtcam import cap
from llmcam.file_manager import list_image_files, list_detection_files
from llmcam.plotting import plot_object 

In [3]:
#| export
# Set up database for information per session
session_messages = {}
session_tools = {}

In [4]:
#| export
# Set up default tools
default_tools = [tool_schema(fn) for fn in (
    capture_youtube_live_frame_and_save, 
    ask_gpt4v_about_image_file,
    detect_objects,
    cap,
    list_image_files,
    list_detection_files,
    plot_object)]

In [5]:
#| export
# Utility functions to manage tools per session
def prepare_handler_schemas(
    session_id: str,  # Session ID to use
    fixup: Callable = None,  # Optional function to fix up the execution
):
    return [
        handler_schema(function, service_name="toolbox_manager", fixup=fixup, session_id=session_id) for \
        function in [add_api_tools, add_function_tools, remove_tools]
    ]

def execute_handler(
    function_name: str,  # Name of the function to execute
    session_id: str,  # Session ID to use
    **kwargs,  # Additional arguments to pass to the function
):
    tools = session_tools[session_id]
    return execute_handler_core(tools, function_name, **kwargs)

In [6]:
#| export
def init_session(session_id: Optional[str] = None):
    if session_id is None or session_id not in session_messages:
        # Initialize tools in session tools and create a session ID
        session_id = str(uuid.uuid4())

        # Add default tools and prepare handler schemas
        session_tools[session_id] = []
        session_tools[session_id].extend(default_tools)
        session_tools[session_id].extend(prepare_handler_schemas(session_id, execute_handler))

        # Initialize messages in session messages
        session_messages[session_id] = []
    
    return session_id

In [7]:
#| export
# Set up the app, including daisyui and tailwind for the chat component
hdrs = (picolink,
        Link(rel="icon", href=f"""{os.getenv("LLMCAM_DATA", "../data")}/favicon.ico""", type="image/png"),
        Script(src="https://cdn.tailwindcss.com"),
        Link(rel="stylesheet", href="https://cdn.jsdelivr.net/npm/daisyui@4.11.1/dist/full.min.css"),
        Script(src="https://unpkg.com/htmx.org"),
        Style("p {color: black;}"),
        Style("li {color: black;}"),
        MarkdownJS(), HighlightJS(langs=['python', 'javascript', 'html', 'css']))
app = FastHTML(hdrs=hdrs, exts="ws")

## Chat components

Basic chat UI components can include Chat Message and a Chat Input. For a Chat Message, the important attributes are the actual message (str) and the role of the message owner (user - boolean value whether the owner is the user, not the AI assistant).

In [8]:
#| export
# Chat message component (renders a chat bubble)
def ChatMessage(
        msg: str,  # Message to display
        user: bool  # Whether the message is from the user or assistant
    ):  # Returns a Div containing the chat bubble
    # Set class to change displayed style of bubble
    content_class = "chat-bubble chat-bubble-primary" if user else ""
    content_class += " marked py-2"
    user_style = "background-color: #fff0c7;"
    return  Div(cls=f"chat chat-end py-4" if user else "py-4")(
                Div('User' if user else 'Assistant', cls="chat-header"),
                Div(
                    msg,
                    cls=content_class,
                    style=user_style if user else "")
            )

In [10]:
show_doc(ChatMessage)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L91){target="_blank" style="float:right; font-size:smaller"}

### ChatMessage

>      ChatMessage (msg:str, user:bool)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| msg | str | Message to display |
| user | bool | Whether the message is from the user or assistant |

For the chat input, set the name for submitting a new message via form.

In [9]:
#| export
# The input field for the user message. Also used to clear the
# input field after sending a message via an OOB swap
def ChatInput():  # Returns an input field for the user message
    return Input(name='msg', id='msg-input', placeholder="Type a message",
                 cls="input input-bordered w-full rounded-l-2xl", 
                 hx_swap_oob='true'  # Re-render the element to remove submitted message
                )

In [12]:
show_doc(ChatInput)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L110){target="_blank" style="float:right; font-size:smaller"}

### ChatInput

>      ChatInput ()

### Action Buttons

Simple actions for creating a new message from the user side.

In [10]:
#| export
def ActionButton(
        session_id: str,  # Session ID to use
        content: str,  # Text to display on the button
        message: str = None  # Message to send when the button is clicked
    ):  # Returns a button with the given content

    return Form(
        ws_send=True,
        hx_ext='ws', ws_connect='/wschat',
    )(
        Hidden(session_id, name="session_id"),
        Hidden(content if message is None else message, name="msg"),
        Button(
            content, 
            cls="btn btn-secondary rounded-2 h-fit", 
            style="background-color: #ffe485; color: black; border-color: #ffe485;")
    )

def ActionPanel(
        session_id: str  # Session ID to use
    ):  # Returns a panel of action buttons
    return Div(
        P("Quick actions", cls="text-lg text-black"),
        ActionButton(session_id, "Introduce your model GPT-4o"),
        ActionButton(session_id,
            "Extract information from a YouTube Live", 
            "Capture and extract information from a YouTube Live. Use the default link."),
        cls="flex flex-col h-fit gap-4 py-4 px-4"
    )

def ToolPanel(
        session_id: str  # Session ID to use
    ):  # Returns a panel of usable tools

    available_services = session_tools.get(session_id, [])

    # Generate list items for each available tool
    items = []
    if available_services:
        for service in available_services:
            service_desc = service['function']['description']
            items.append(Li(f"{service_desc}", cls="text-sm text-black"))
    else:
        items.append(Li("No services available", cls="text-sm italic text-gray-500"))

    return Div(
        P("Available Tools", cls="text-lg text-black"),
        Ul(*items, cls="list-disc list-inside px-6", style="max-height: 200px; overflow-y:auto;"),
        id="toollist",
        cls="flex flex-col h-fit gap-4 py-4 px-4"
    )

In [11]:
show_doc(ActionButton)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L117){target="_blank" style="float:right; font-size:smaller"}

### ActionButton

>      ActionButton (session_id:str, content:str, message:str=None)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| session_id | str |  | Session ID to use |
| content | str |  | Text to display on the button |
| message | str | None | Message to send when the button is clicked |

In [12]:
show_doc(ActionPanel)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L135){target="_blank" style="float:right; font-size:smaller"}

### ActionPanel

>      ActionPanel (session_id:str)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| session_id | str | Session ID to use |

## Router

### Home page
The home page should contain our message list and the Chat Input. The main page can be extracted by accessing the index (root) endpoint.

In [13]:
#| export
scroll_script = Script("""
  // Function to scroll to the bottom of an element
  function scrollToBottom(element) {
    element.scrollTop = element.scrollHeight;
  }

  // Reference the expanding element
  const expandingElement = document.getElementById('chatlist');

  // Observe changes to the element's content and scroll down automatically
  const observer = new MutationObserver(() => {
    scrollToBottom(expandingElement);
  });

  // Start observing the expanding element for changes
  observer.observe(expandingElement, { childList: true, subtree: true });
""")

In [14]:
#| export
title_script = Script("""
    // Function to set the title of the page
    document.title = "LLMCAM";
""")

In [55]:
noti_script = Script("""
    let noti = null
    document.body.addEventListener('htmx:load', function(event) {
        if (event.target.id === "notification") {
            let htDivElement = event.detail.elt; // Extract the HtDiv element

            // Find the input element inside the HtDiv and extract its value
            let inputElement = htDivElement.querySelector('input');
            if (inputElement) {
                let inputValue = inputElement.value;
                alert("New notification has been loaded with value: " + inputValue);
            } else {
                console.log("Input element not found.");
            }
        }
    });
""")

In [25]:
def NotiMessage(message = "No message"):
    return Hidden(message, id="notification", cls="text-black")

In [56]:
#| export
@app.get('/')
async def index(session):
    # Initialize the session
    session_id = init_session(session_id=session.get('session_id'))
    
    # Set up the chat UI
    sidebar = Div(
        ActionPanel(session_id=session_id),
        ToolPanel(session_id=session_id),
        Button("Notification", id="connect-btn", ws_send=True, hx_ext='ws', ws_connect='/wsnoti', cls="btn btn-primary rounded-2 h-fit", style="background-color: #ffe485; color: black; border-color: #ffe485;"),
        NotiMessage(),
        P("Conversations", cls="text-lg text-black px-4"),
        cls="w-[30vw] flex flex-col p-0",
        style="background-color: White;"
    )
    page =  Div(cls="w-full flex flex-col p-0")(  # Main page
        Form(
            ws_send=True,
            hx_ext='ws', ws_connect='/wschat',
            cls="w-full flex flex-col px-24 h-[100vh]"
        )(
            Hidden(session_id, name="session_id"),
            # The chat list
            Div(id="chatlist", cls="chat-box overflow-y-auto flex-1 w-full mt-10 p-4")(
                # One initial message from AI assistant
                ChatMessage("Hello! I'm a chatbot. How can I help you today?", False),
            ),
            # Input form
            Div(cls="h-fit mb-5 mt-5 flex space-x-2 mt-2 p-4")(
                Group(
                    ChatInput(), 
                    Button("Send", cls="btn btn-primary rounded-r-2xl", style="background-color: #ffe485;"))
            ),
            scroll_script
        )   
    )
    return Main(
        noti_script,
        title_script,
        sidebar,
        page, 
        title="Chatbot",
        data_theme="lemonade", 
        cls="h-[100vh] w-full relative flex flex-row items-stretch overflow-hidden transition-colors z-0 p-0",)

### Notification websockets

In [42]:
import asyncio

@app.ws('/wsnoti')
async def wsnoti(ws, send):
    while True:
        message = f"Server Time: {asyncio.get_event_loop().time()}"
        await ws.send_text(message)
        await send(Div(NotiMessage(message), id="notification", hx_swap_oob="true"))
        await asyncio.sleep(10)

### Chat websockets

When connecting to websockets for chat, this function should:

- Extract the new and all previous chat history  
- Prompt & get answers from ChatGPT from all these messages  
- Return a new ChatMessage

In [19]:
#| export
# On websocket disconnect, remove the session ID from the session messages and tools
def chat_disconnect(ws):
    """Remove session ID from session messages and tools on websocket disconnect"""
    session_id = ws.scope.get("session_id")
    if session_id in session_messages:
        del session_messages[session_id]
    if session_id in session_tools:
        del session_tools[session_id]

In [20]:
#| export
# The chatbot websocket handler
@app.ws('/wschat', disconn=chat_disconnect)
async def wschat(ws, msg: str, send, session_id: str):
    # Initialize the session
    session_id = init_session(session_id=session_id)

    # Set the session ID in the websocket scope
    ws.scope["session_id"] = session_id

    # Set up the global variables
    global session_tools
    global execute_handler
    
    # Create chat messages from the provided contents and roles
    messages = session_messages.get(session_id, [])
    if len(messages) == 0:
        messages.append(
            form_msg(
                "system", 
"You are a helpful assistant. Use the supplied tools to assist the user. \
If asked to show or display an image or plot, do it by embedding its path starting with \
`../data/<filename>` in Markdown syntax."
            ))
    messages.append(form_msg("user", msg))
    await send(
        Div(ChatMessage(
            messages[-1]["content"],
            messages[-1]["role"] == "user"), 
        hx_swap_oob='beforeend', id="chatlist"))
    
    await send(ChatInput())  # Clear the input field
    
    # Add the user's message to the chat history
    complete(messages, session_tools[session_id])
    await send(Div(ChatMessage(
            messages[-1]["content"],
            messages[-1]["role"] == "user"), hx_swap_oob='beforeend', id="chatlist"))
    
    await send(Div(ToolPanel(session_id=session_id), hx_swap_oob='true', id='toollist'))
    return

### Static files

In case the user needs to display images, serves files from directory `../data`.

In [22]:
#| export
# Serve files from the 'data' directory
@app.get("/data/{file_name:path}")
async def get_file(file_name: str):
    """Serve files dynamically from the 'data' directory."""
    data_path = os.getenv("LLMCAM_DATA", "../data")
    file_path = Path(data_path) / file_name
    if file_path.exists():
        return FileResponse(file_path)
    return {"error": f"File '{file_name}' not found"}

## Runner

In addition to the main app, an utility function is implemented to run the app just by importing and executing this function to a Python file.

In [ ]:
#| export
import asyncio
import time

def llmcam_chatbot(
        host="0.0.0.0",  # The host to listen on
        port=5001,  # The port to listen on
    ):
    # Import app from chat_ui base module
    from llmcam.chat_ui import app

    # Initialize session tools and execute handler
    session_tools = {}
    globals()["session_tools"] = session_tools

    def execute_handler(
        function_name: str,  # Name of the function to execute
        session_id: str,  # Session ID to use
        **kwargs,  # Additional arguments to pass to the function
    ):
        tools = session_tools[session_id]
        return execute_handler_core(tools, function_name, **kwargs)
    
    globals()["execute_handler"] = execute_handler
    
    server = uvicorn.Server(uvicorn.Config(app, host=host, port=port))
    async def async_run_server(server): await server.serve()
    asyncio.run(async_run_server(server))

In [ ]:
show_doc(llmcam_chatbot)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L304){target="_blank" style="float:right; font-size:smaller"}

### llmcam_chatbot

>      llmcam_chatbot (host='0.0.0.0', port=5001)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| host | str | 0.0.0.0 | The host to listen on |
| port | int | 5001 | The port to listen on |

For running while testing with Jupyter notebook, use the `JupyUvi` in `fasthtml` to run in separate thread.

In [ ]:
#| eval: false
from fasthtml.jupyter import *

server = JupyUvi(app=app)

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/nghivo/miniconda3/envs/llmcam/lib/python3.12/site-packages/uvicorn/protocols/websockets/websockets_impl.py", line 330, in asgi_send
    await self.send(data)  # type: ignore[arg-type]
    ^^^^^^^^^^^^^^^^^^^^^
  File "/home/nghivo/miniconda3/envs/llmcam/lib/python3.12/site-packages/websockets/legacy/protocol.py", line 628, in send
    await self.ensure_open()
  File "/home/nghivo/miniconda3/envs/llmcam/lib/python3.12/site-packages/websockets/legacy/protocol.py", line 929, in ensure_open
    raise self.connection_closed_exc()
websockets.exceptions.ConnectionClosedOK: received 1001 (going away); then sent 1001 (going away)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/nghivo/miniconda3/envs/llmcam/lib/python3.12/site-packages/starlette/websockets.py", line 85, in send
    await self._send(message)
  File "/home/nghivo/miniconda

In [30]:
#| eval: false
server.stop()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()